# Export station coords

Export station coordinates to a file to that it is readily available when we perform interpolation.
This is better than rerunning the query to the mongodb every time, because it's kind of expensive to 
look at 2 years of data to find all the possible stations.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import pymongo
import random

In [ ]:
MONGO_URL = 'localhost'
MONGO_PORT = 27017
USERNAME = None
PASSWORD = None
ADMIN_DB = 'admin'
COLLECTION = 'iem'
DB = 'smc01_raw_obs'

In [ ]:
begin_date = datetime.datetime(2019, 1, 1, tzinfo=datetime.timezone.utc)
end_date = datetime.datetime(2021, 1, 1, tzinfo=datetime.timezone.utc)

In [ ]:
n_days = end_date - begin_date

In [ ]:
days = list(range(n_days.days))

In [ ]:
random.shuffle(days)

In [ ]:
days_sample

In [ ]:
mongo_client = pymongo.MongoClient(host=MONGO_URL, port=MONGO_PORT, username=USERNAME, password=PASSWORD, authSource=ADMIN_DB)
db = mongo_client[DB]
collection = db[COLLECTION]

In [ ]:
for d in days[:1]:
    query_filter = {
    'valid': {
        '$gte': begin_date + datetime.timedelta(days=d),
        '$lt': begin_date + datetime.timedelta(days=d+1)
    }}
    
    response = collection.distinct('station', filter=query_filter)

In [ ]:
stations = set(response)

In [ ]:
len(stations)

In [ ]:
query_filter = {
    'valid': {
        '$gte': begin_date,
        '$lt': end_date
}}

In [ ]:
response = collection.distinct('station', filter=query_filter)

In [ ]:
for station in stations:
    one_obs = collection.find_one({'station': station})
    station_infos.append({
        'station': station,
        'lat': one_obs['lat'],
        'lon': one_obs['lon'],
    })
